# Dataset ($\Omega_b, \theta_*$)  - CAMB

In [2]:
import camb
import numpy as np
import csv

In [3]:
# means Planck 2018 ------------------

omg_b = 0.0224  # this is actually Omg_b*(h^2)           0.015-0.03
omg_cdm = 0.120 # this is actually Omg_cdm*(h^2)          0.05-0.2

# no theta_star   
tau = 0.054
n_s = 0.965
A_s = 2.0968e-9
omg_k=0.001 # -0.2 +0.2   

h = 67.4/100  # 30- 120

# sigmas Planck 2018 ----- 


sigma_omg_b = 0.0001    #
sigma_omg_cdm = 0.001   #      
sigma_tau = 0.007
sigma_n_s = 0.004
sigma_A_s = 2.9e-11
sigma_omg_k = 0.002

sigma_h = 0.5/100

PLANCK2018_CLASS_names = ['omega_b', 'omega_cdm', 'tau_reio', 'n_s', 'A_s', 'h', 'Omega_k']
PLANCK2018_means = [omg_b, omg_cdm, tau, n_s, A_s, h, omg_k]
PLANCK2018_sigmas = [sigma_omg_b, sigma_omg_cdm, sigma_tau, sigma_n_s, sigma_A_s, sigma_h, sigma_omg_k]  
                
PLANCK2018_params = [[i,j,k] for i,j,k in zip(PLANCK2018_CLASS_names, PLANCK2018_means, PLANCK2018_sigmas)]


In [4]:
def parameter_array(start, end, N):
    param_arr = np.linspace(start, end , N)
    return param_arr

#### alternative sintax (in case theta star not found in derived)

In [16]:
%%time
# Set up parameters. BEST WAY TO DO IT
pars = camb.CAMBparams()  # <--- this is the correct class name
pars.set_cosmology( H0=67.36,               # Hubble parameter today
    ombh2=0.02237,          # Baryon density parameter
    omch2=0.1200,           # Cold dark matter density parameter
    mnu=0.06,               # Minimal mass for neutrinos (one massive)
    omk=0.0,                # Curvature — this is what you'll vary in MCMC
    tau=0.0544)             # Reionization optical depth

pars.InitPower.set_params(As=2.100e-9, ns=0.9649)

# Only calculate scalar TT power spectrum
pars.WantTensors = False
pars.WantScalars = True
pars.WantCls = True
pars.DoLensing = False
pars.set_for_lmax(3000, lens_potential_accuracy=0)
pars.OutputCls = ['T']

# Run CAMB
results = camb.get_results(pars)
cls = results.get_cmb_power_spectra(pars, CMB_unit='muK')
TT = cls['total'][:, 0]  # TT spectrum

derived = results.get_derived_params()

#theta_star = derived['theta_star']    # THIS IS WHAT YOU WANT
rs_star    = derived['rstar']          # sound horizon at decoupling
DA_star    = derived['DAstar']         # comoving angular diameter distance

#print("theta_* =", theta_star)
print("r_s(z*) =", rs_star)
print("D_A(z*) =", DA_star)
print("Theta* =", rs_star/DA_star)

r_s(z*) = 144.44041891870265
D_A(z*) = 13.87278675949587
Theta* = 10.411781095087743
CPU times: user 2.82 s, sys: 32.2 ms, total: 2.85 s
Wall time: 437 ms


In [17]:
pars = camb.CAMBparams()
# set static things once

for params in samples:
    pars.set_cosmology(H0=..., ombh2=..., omch2=..., tau=...)
    results = camb.get_results(pars)
    theta = results.get_derived_params()['theta_star']

NameError: name 'samples' is not defined

In [ ]:
def dataset_grid(param_ranges, N_per_param):
    param_arrays = [parameter_array(start, end, N_per_param) for start, end in param_ranges]
    mesh = np.meshgrid(*param_arrays)
    param_grid = np.vstack([m.flatten() for m in mesh]).T
    return param_grid

------
#### defs

In [5]:
def compute_theta_star(omg_bh2, omg_ch2, h, omg_k):
    
    pars = camb.CAMBparams() # maybe initialize CAMB parameters outside the function for speed

    # Set cosmology
    pars.set_cosmology(ombh2=omg_bh2, omch2=omg_ch2, H0=100*h, omk = omg_k, tau=0.054)

    # Power spectrum settings (won't be used)
    pars.InitPower.set_params(As=2.0968e-9, ns=0.965)

    # SPEED TRICK 1: Do NOT compute any CMB power spectra
    pars.set_for_lmax(lmax=0, lens_potential_accuracy=0)
    pars.WantDerivedParameters = True  # needed for theta_star

    # SPEED TRICK 2: Use low-accuracy background settings
    pars.set_accuracy(
        AccuracyBoost=0.1,
        lAccuracyBoost=0.1,
        lSampleBoost=0.1
    )
    
    # SPEED TRICK 3: Only compute background and recombination
    pars.DoLateRadTruncation = True
    pars.DoLensing = False
    
    # Run CAMB
    results = camb.get_results(pars)

    #    # Extract θ*
    #    theta_star = results.get_derived_params()['theta_star']
    derived = results.get_derived_params()

    #theta_star = derived['theta_star']    # THIS IS WHAT WE WANT
    rs_star    = derived['rstar']          # sound horizon at decoupling  - in Mpc
    DA_star    = derived['DAstar']         # comoving angular diameter distance - in Gpc
    theta_star = 100*rs_star/ (DA_star * 1000.)# 100*theta_star dimensionless

    #print("theta_* =", theta_star)
    print("r_s(z*) =", rs_star)
    print("D_A(z*) =", DA_star)
    print("Theta* =", theta_star)
    return theta_star


------
#### ll

In [6]:
compute_theta_star(0.0224, 0.120, 67.4/100., 0.001)

: 

In [ ]:
def dataset_grid(li1=[0.015,0.03], li2=[0.05,0.2], li3 = [30.0/100,120.0/100], li4=[-0.2,0.2 ], N_per_param=2):
    rows = []
    for o1 in parameter_array(li1[0], li1[1], N_per_param):
        for o2 in parameter_array(li2[0], li2[1], N_per_param):
            for o3 in parameter_array(li3[0], li3[1], N_per_param):
                for o4 in parameter_array(li4[0], li4[1], N_per_param):
                    print(f"Trying: omg_bh2: {o1}, omg_ch2: {o2}, H0: {o3}, omg_kh2: {o4}")
                    theta_star = compute_theta_star(o1, o2, o3, o4)
                    rows.append(( o1, o2, o3, o4, theta_star))
    # write CSV
    out_csv = "GRID4_theta_s"+'_N'+str(N_per_param)+".csv"
    with open(out_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['omega_b', 'omega_cdm', 'h', 'Omega_k', '100*theta_s'])
        for r in rows:
            writer.writerow(r)

    print(f"Saved {len(rows)} rows to {out_csv}")
    return 

In [19]:
dataset_grid(li1=[0.015,0.03], li2=[0.05,0.2], li3 = [30./100.,120.0/100.], li4=[-0.001,0.001], N_per_param=1)

Trying: omg_bh2: 0.015, omg_ch2: 0.05, H0: 0.3, omg_kh2: -0.001


NameError: name 'compute_theta_star' is not defined